<a href="https://colab.research.google.com/github/sergiusz28/dw_matrix_road_sign/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from skimage import color, exposure

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
cd '/content/drive/My Drive/Colab Notebooks/dwmatrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dwmatrix/matrix_three/dw_matrix_road_sign
ERROR! Session/line number was not unique in database. History logging moved to new session 66


In [8]:

train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

x_train, y_train = train['features'], train['labels']
x_test, y_test = test['features'], test['labels']

if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = x_train.shape[1:]
num_classes = y_train.shape[1]

ERROR! Session/line number was not unique in database. History logging moved to new session 67


In [0]:
def train_model(model, x_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%sS'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(x_train,
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (x_train, y_train)),
            callbacks= [tensorboard_callback]
            )
  return model

def predict(model_trained, x_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(x_test)
  y_pred = np.argmax(y_pred_prob, axis=1)
  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same' ),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

In [15]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, x_train, y_train)

predict(model_trained, x_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 14ms/step - loss: 2.8402 - accuracy: 0.2404 - val_loss: 1.1463 - val_accuracy: 0.6359
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.8709 - accuracy: 0.7209 - val_loss: 0.1953 - val_accuracy: 0.9393
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3486 - accuracy: 0.8926 - val_loss: 0.0840 - val_accuracy: 0.9760
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2144 - accuracy: 0.9361 - val_loss: 0.0405 - val_accuracy: 0.9882
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1760 - accuracy: 0.9492 - val_loss: 0.0271 - val_accuracy: 0.9926


0.9682539682539683

In [22]:
model_trained.evaluate(x_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1551 - accuracy: 0.9683


0.9682539701461792

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same' ),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same' ),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),
    
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),
    
    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%sS'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(x_train,
            y_train,
            batch_size=128,#int(params.get('batch_size', 128)),
            epochs=5,
            verbose=0
            )
  score = model.evaluate(x_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  return {'loss':-accuracy, 'status': STATUS_OK, 'model': model}

In [40]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two',0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 180.0, 'dropout_cnn_block_one': 0.305406329575419, 'dropout_cnn_block_three': 0.30369260452167773, 'dropout_cnn_block_two': 0.4938594477109193, 'dropout_dense_block_one': 0.6698802771064422, 'dropout_dense_block_two': 0.4492337501302737}
accuracy=0.9204081892967224
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.35774013751374256, 'dropout_cnn_block_three': 0.4830246005688598, 'dropout_cnn_block_two': 0.43308906057569774, 'dropout_dense_block_one': 0.30920851170244573, 'dropout_dense_block_two': 0.6570597174723776}
accuracy=0.9498866200447083
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.32339626000192423, 'dropout_cnn_block_three': 0.4099649079006216, 'dropout_cnn_block_two': 0.48012957167916276, 'dropout_dense_block_one': 0.5092430221906734, 'dropout_dense_block_two': 0.5896592582345731}
accuracy=0.9514739513397217
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.3585693119842477, 'dropout_cnn_block_three': 0.47279503073290463, 'dropout_cnn_block_two': 0.3119303817